In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import sqlite3
from sqlalchemy import create_engine # database connection
import csv
import os
warnings.filterwarnings("ignore")
import datetime as dt
import numpy as np
from nltk.corpus import stopwords
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
#from sklearn.metrics.classification import accuracy_score, log_loss
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from scipy.sparse import hstack
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold 
#"from sklearn.model_selection import StratifiedKFold" instead of  "from sklearn.cross_validation import StratifiedKFold"
from collections import Counter, defaultdict
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import math
from sklearn.metrics import normalized_mutual_info_score
from sklearn.ensemble import RandomForestClassifier



from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import StackingClassifier

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve, auc, roc_curve

# Data Loading

In [ ]:
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/02. Machine Learning/0/GSE188678_COVID_host_classifier_gene_counts.csv", index_col=0)
#data.drop(index = 0, inplace = True)
print(data.shape)
data.head()

(19940, 318)


,RR057e_00753,RR057e_00751,RR057e_00726,RR057i_00157,RR057i_00167,RR057e_00749,RR057i_00076,RR057e_00754,RR057e_00735,RR057e_00727,...,RR057e_00878,RR057e_00879,RR057e_00880,RR057e_00881,RR057e_00882,RR057e_00883,RR057e_00884,RR057e_00885,RR057e_00886,RR057e_00887
Gene_Symbol,,,,,,,,,,,,,,,,,,,,,
Target,infection: SC2,infection: SC2,infection: SC2,infection: SC2,infection: SC2,infection: SC2,infection: SC2,infection: SC2,infection: SC2,infection: SC2,...,infection: no_virus,infection: other_virus,infection: no_virus,infection: no_virus,infection: no_virus,infection: no_virus,infection: no_virus,infection: no_virus,infection: other_virus,infection: no_virus
ENSG00000000003,1252,151,385,6,431,657,359,271,23,63,...,99,43,25,988,651,736,690,328,443,450
ENSG00000000005,10,0,5,0,0,9,6,7,0,9,...,0,0,0,0,0,0,0,0,0,0
ENSG00000000419,603,160,276,16,282,326,353,126,23,50,...,34,127,812,315,317,290,432,75,7,462
ENSG00000000457,199,44,99,5,143,121,117,36,15,7,...,11,27,81,33,113,139,105,5,30,175


In [ ]:
df1 = data.T
df1.head()

Gene_Symbol,Target,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,...,ENSG00000288436,ENSG00000288460,ENSG00000288520,ENSG00000288529,ENSG00000288534,ENSG00000288547,ENSG00000288550,ENSG00000288558,ENSG00000288564,ENSG00000288570
RR057e_00753,infection: SC2,1252,10,603,199,182,192,766,479,9783,...,0,0,0,0,75,15,0,27,0,0
RR057e_00751,infection: SC2,151,0,160,44,20,21,126,57,747,...,0,0,0,0,5,0,2,4,7,0
RR057e_00726,infection: SC2,385,5,276,99,89,54,562,254,1451,...,0,0,0,0,22,1,0,15,2,0
RR057i_00157,infection: SC2,6,0,16,5,1,7,21,6,32,...,0,0,0,0,0,0,0,2,0,0
RR057i_00167,infection: SC2,431,0,282,143,90,539,672,252,1395,...,0,0,0,0,29,0,0,5,5,0


In [ ]:
print(df1.isna().sum().max())


0


In [ ]:
df1['Target'].value_counts()

infection: no_virus       169
infection: SC2             90
infection: other_virus     59
Name: Target, dtype: int64

In [ ]:
# replacing values
df1['Target'] = df1['Target'].replace(['infection: no_virus', 'infection: SC2', 'infection: other_virus'], [0, 1, 2])
df1.head()

Gene_Symbol,Target,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,...,ENSG00000288436,ENSG00000288460,ENSG00000288520,ENSG00000288529,ENSG00000288534,ENSG00000288547,ENSG00000288550,ENSG00000288558,ENSG00000288564,ENSG00000288570
RR057e_00753,1,1252,10,603,199,182,192,766,479,9783,...,0,0,0,0,75,15,0,27,0,0
RR057e_00751,1,151,0,160,44,20,21,126,57,747,...,0,0,0,0,5,0,2,4,7,0
RR057e_00726,1,385,5,276,99,89,54,562,254,1451,...,0,0,0,0,22,1,0,15,2,0
RR057i_00157,1,6,0,16,5,1,7,21,6,32,...,0,0,0,0,0,0,0,2,0,0
RR057i_00167,1,431,0,282,143,90,539,672,252,1395,...,0,0,0,0,29,0,0,5,5,0


In [ ]:
df1.drop(df1.index[df1['Target'] == 2], inplace=True)
df1['Target'].value_counts()

0    169
1     90
Name: Target, dtype: int64

In [ ]:
labels = df1.Target
labels

RR057e_00753    1
RR057e_00751    1
RR057e_00726    1
RR057i_00157    1
RR057i_00167    1
               ..
RR057e_00882    0
RR057e_00883    0
RR057e_00884    0
RR057e_00885    0
RR057e_00887    0
Name: Target, Length: 259, dtype: int64

In [ ]:
# Drop the label 'Locations'
#df1.drop(labels='Target', axis=1, inplace = True)
df1.head()

Gene_Symbol,Target,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,...,ENSG00000288436,ENSG00000288460,ENSG00000288520,ENSG00000288529,ENSG00000288534,ENSG00000288547,ENSG00000288550,ENSG00000288558,ENSG00000288564,ENSG00000288570
RR057e_00753,1,1252,10,603,199,182,192,766,479,9783,...,0,0,0,0,75,15,0,27,0,0
RR057e_00751,1,151,0,160,44,20,21,126,57,747,...,0,0,0,0,5,0,2,4,7,0
RR057e_00726,1,385,5,276,99,89,54,562,254,1451,...,0,0,0,0,22,1,0,15,2,0
RR057i_00157,1,6,0,16,5,1,7,21,6,32,...,0,0,0,0,0,0,0,2,0,0
RR057i_00167,1,431,0,282,143,90,539,672,252,1395,...,0,0,0,0,29,0,0,5,5,0


In [ ]:
X, y = df1.drop(columns=["Target"]), df1["Target"]

In [ ]:
X.head()

Gene_Symbol,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,...,ENSG00000288436,ENSG00000288460,ENSG00000288520,ENSG00000288529,ENSG00000288534,ENSG00000288547,ENSG00000288550,ENSG00000288558,ENSG00000288564,ENSG00000288570
RR057e_00753,1252,10,603,199,182,192,766,479,9783,160,...,0,0,0,0,75,15,0,27,0,0
RR057e_00751,151,0,160,44,20,21,126,57,747,51,...,0,0,0,0,5,0,2,4,7,0
RR057e_00726,385,5,276,99,89,54,562,254,1451,90,...,0,0,0,0,22,1,0,15,2,0
RR057i_00157,6,0,16,5,1,7,21,6,32,6,...,0,0,0,0,0,0,0,2,0,0
RR057i_00167,431,0,282,143,90,539,672,252,1395,145,...,0,0,0,0,29,0,0,5,5,0


In [ ]:
y

RR057e_00753    1
RR057e_00751    1
RR057e_00726    1
RR057i_00157    1
RR057i_00167    1
               ..
RR057e_00882    0
RR057e_00883    0
RR057e_00884    0
RR057e_00885    0
RR057e_00887    0
Name: Target, Length: 259, dtype: int64

In [ ]:
Y = LabelEncoder().fit_transform(y)
#np.set_printoptions(threshold=np.inf)
Y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

# Data Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =  train_test_split(X,y,test_size = 0.25, random_state= 42)

In [ ]:
from plotly.offline import init_notebook_mode, iplot
def complete_test_2D(X, Y, plot_name = ''):
    Small_df = pd.DataFrame(data = X, columns = ['C1', 'C2'])
    Small_df = pd.concat([Small_df, df1['Target']], axis = 1)
    Small_df['Target'] = LabelEncoder().fit_transform(Small_df['Target'])
    forest_test(X, Y)
    data = []
    for clas, col, name in zip((1, 0), ['red', 'darkblue'], ['Covid', 'no_virus']):

        trace = dict(
            type='scatter',
            x= Small_df.loc[Small_df['Target'] == clas, 'C1'],
            y= Small_df.loc[Small_df['Target'] == clas, 'C2'],
            mode= 'markers',
            name= name,
            marker=dict(
                color=col,
                size=12,
                line=dict(
                    color='rgba(217, 217, 217, 0.14)',
                    width=0.5),
                opacity=0.8)
        )
        data.append(trace)

    layout = dict(
            title= plot_name + ' 2D Dimensionality Reduction',
            xaxis=dict(title='C1', showline=False),
            yaxis=dict(title='C2', showline=False)
    )
    fig = dict(data=data, layout=layout)
    iplot(fig)

In [ ]:
def complete_test_3D(X, Y, plot_name = ''):
    Small_df = pd.DataFrame(data = X, columns = ['C1', 'C2', 'C3'])
    Small_df = pd.concat([Small_df, df1['Target']], axis = 1)
    Small_df['Target'] = LabelEncoder().fit_transform(Small_df['Target'])
    forest_test(X, Y)
    data = []
    for clas, col, name in zip((1, 0), ['red', 'darkblue'], ['Covid', 'no_virus']):

        trace = dict(
            type='scatter3d',
            x= Small_df.loc[Small_df['Target'] == clas, 'C1'],
            y= Small_df.loc[Small_df['Target'] == clas, 'C2'],
            z= Small_df.loc[Small_df['Target'] == clas, 'C3'],
            mode= 'markers',
            name= name
        )
        data.append(trace)

    layout = {
        "scene": {
          "xaxis": {
            "title": "C1", 
            "showline": False
          }, 
          "yaxis": {
            "title": "C2", 
            "showline": False
          }, 
          "zaxis": {
            "title": "C3", 
            "showline": False
          }
        }, 
        "title": plot_name + ' 3D Dimensionality Reduction'
    }
    fig = dict(data=data, layout=layout)
    iplot(fig)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier

X = StandardScaler().fit_transform(X)

In [ ]:
def forest_test(X, Y):
    X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, test_size = 0.30, random_state = 101)
    start = time.process_time()
    trainedforest = RandomForestClassifier(n_estimators=125).fit(X_Train,Y_Train)
    print("Time: ", time.process_time() - start)
    predictionforest = trainedforest.predict(X_Test)
    print(confusion_matrix(Y_Test,predictionforest))
    print(classification_report(Y_Test,predictionforest))

In [ ]:
forest_test(X, y)

Time:  1.2792986670000008
[[49  2]
 [ 4 23]]
              precision    recall  f1-score   support

           0       0.92      0.96      0.94        51
           1       0.92      0.85      0.88        27

    accuracy                           0.92        78
   macro avg       0.92      0.91      0.91        78
weighted avg       0.92      0.92      0.92        78

